In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12685843360914856040, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9214062756
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 16266123730375111453
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [3]:
import os
import numpy as np
np.random.seed(777)

import keras.backend as K
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.datasets import cifar10, mnist
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras import backend as K

from mlxtend.classifier import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import accuracy_score

from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [4]:
# pip install mlxtend

In [5]:
img_height, img_width = 64, 64
batch_size = 4
epochs = 5

input_shape = (img_height, img_width, 3)
num_classes = 2
nb_train_samples = 386
nb_validation_samples = 199
nb_test_samples = 155

In [6]:
train_dir = 'data_reduced/train/'
validation_dir = 'data_reduced/validation'
test_dir = 'data_reduced/test'

In [7]:
random_seed = np.random.seed(777)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size = 1,
    seed = random_seed,
    shuffle = True,
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    batch_size = 1,
    seed = random_seed,
    shuffle = True,
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=1,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 386 images belonging to 2 classes.
Found 199 images belonging to 2 classes.
Found 155 images belonging to 2 classes.


In [8]:
X_train, y_train = [], []
for _ in tqdm(range(nb_train_samples)):
    x, y = train_generator.next()
    X_train.append(x[0])
    y_train.append(y[0])
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
y_train = np.argmax(y_train, axis=1)
# np.save('data/npy/X_train.npy', X_train)
# np.save('data/npy/y_train.npy', y_train)

100%|███████████████████████████████████████████████████████████████████████████████| 386/386 [00:00<00:00, 472.93it/s]


In [9]:
# X_train = np.array([x.flatten() for x in X_train])

In [10]:
X_validation, y_validation = [], []
for _ in tqdm(range(nb_validation_samples)):
    x_val, y_val = validation_generator.next()
    X_validation.append(x_val[0])
    y_validation.append(y_val[0])
X_validation = np.asarray(X_validation)
y_validation = np.asarray(y_validation)
y_validation = np.argmax(y_validation, axis=1)
# np.save('data/npy/X_validation.npy', X_validation)
# np.save('data/npy/y_validation.npy', y_validation)

100%|███████████████████████████████████████████████████████████████████████████████| 199/199 [00:00<00:00, 550.29it/s]


In [11]:
# X_validation = np.array([x.flatten() for x in X_validation])

In [12]:
X_test, y_test = [], []
for _ in tqdm(range(nb_test_samples)):
    x_t, y_t = test_generator.next()
    X_test.append(x_t[0])
    y_test.append(y_t[0])
X_test = np.asarray(X_test)
y_test = np.asarray(y_test)
y_test = np.argmax(y_test, axis=1)
# np.save('data/npy/X_test.npy', X_test)
# np.save('data/npy/y_test.npy', y_test)

100%|███████████████████████████████████████████████████████████████████████████████| 155/155 [00:00<00:00, 538.73it/s]


In [13]:
# X_test = np.array([x.flatten() for x in X_test])

In [14]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_validation shape:", X_validation.shape)
print("y_validation shape:", y_validation.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)
# plt.imshow(X_train[0])
# plt.show()

X_train shape: (386, 64, 64, 3)
y_train shape: (386,)
X_validation shape: (199, 64, 64, 3)
y_validation shape: (199,)
X_test shape: (155, 64, 64, 3)
y_test shape: (155,)


____

In [15]:
class AccuracyHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))

In [16]:
def build_model1():
    model = Sequential()
    model.add(Conv2D(28, kernel_size=(5, 5), strides=(1, 1),
                     activation='relu',
                     input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.SGD(lr=0.01),
                  metrics=['accuracy'])

    return model

def build_model2():
    model = Sequential()
    model.add( Conv2D(28, kernel_size=(4,4), strides=(2,2), activation='relu',input_shape=input_shape) )
    model.add( AveragePooling2D(pool_size=(2,2), strides=(2,2)) )
    model.add( Conv2D(64, kernel_size=(4,4), strides=(2,2), activation='relu') )
    model.add( AveragePooling2D(pool_size=(2,2), strides=(2,2)) )
    model.add(Flatten())
    model.add(Dense(40, activation='relu'))
    model.add(Dense(40, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.SGD(lr=0.01),
                  metrics=['accuracy'])
    return model

In [17]:
history = AccuracyHistory()

In [18]:
model1 = KerasClassifier( build_fn=build_model1,
                          batch_size=batch_size,
                          epochs=epochs,
                          verbose=2)
#                           validation_data=(X_validation, y_validation))
#                              callbacks=[history])


model2 = KerasClassifier( build_fn=build_model2,
                          batch_size=batch_size,
                          epochs=epochs,
                          verbose=2)
#                           validation_data=(X_validation, y_validation))
#                              callbacks=[history])


stacked_model = StackingClassifier( classifiers=[model1,model2],
                                    meta_classifier=LogisticRegression( penalty='l2' ),
                                    use_probas=True )

for m,name in zip((model1, model2, stacked_model), ('CNN 1', 'CNN 2', 'Stacked Model')):
    kfold = KFold(n_splits=10, shuffle=True)
    scores = cross_val_score( m, X_train, y_train, cv=kfold, scoring='accuracy' )
    print( '{}: {}'.format(name,scores.mean()) )


Epoch 1/5
 - 4s - loss: 0.3358 - acc: 0.8098
Epoch 2/5
 - 1s - loss: 0.2478 - acc: 0.8847
Epoch 3/5
 - 1s - loss: 0.2010 - acc: 0.9164
Epoch 4/5
 - 1s - loss: 0.1723 - acc: 0.9280
Epoch 5/5
 - 1s - loss: 0.1541 - acc: 0.9366
Epoch 1/5
 - 2s - loss: 0.3581 - acc: 0.8040
Epoch 2/5
 - 1s - loss: 0.2467 - acc: 0.8818
Epoch 3/5
 - 1s - loss: 0.1910 - acc: 0.9280
Epoch 4/5
 - 1s - loss: 0.1534 - acc: 0.9366
Epoch 5/5
 - 1s - loss: 0.1360 - acc: 0.9481
Epoch 1/5
 - 2s - loss: 0.3411 - acc: 0.8098
Epoch 2/5
 - 1s - loss: 0.2327 - acc: 0.8905
Epoch 3/5
 - 1s - loss: 0.1979 - acc: 0.9222
Epoch 4/5
 - 1s - loss: 0.1763 - acc: 0.9308
Epoch 5/5
 - 1s - loss: 0.1624 - acc: 0.9337
Epoch 1/5
 - 1s - loss: 0.3337 - acc: 0.8012
Epoch 2/5
 - 1s - loss: 0.2307 - acc: 0.8876
Epoch 3/5
 - 1s - loss: 0.1788 - acc: 0.9308
Epoch 4/5
 - 1s - loss: 0.1463 - acc: 0.9424
Epoch 5/5
 - 1s - loss: 0.1363 - acc: 0.9481
Epoch 1/5
 - 2s - loss: 0.3220 - acc: 0.8329
Epoch 2/5
 - 1s - loss: 0.2392 - acc: 0.8934
Epoch 3/5


Epoch 2/5
 - 1s - loss: 0.2283 - acc: 0.8879
Epoch 3/5
 - 1s - loss: 0.1869 - acc: 0.9253
Epoch 4/5
 - 1s - loss: 0.1588 - acc: 0.9339
Epoch 5/5
 - 1s - loss: 0.1265 - acc: 0.9569
Epoch 1/5
 - 2s - loss: 0.5031 - acc: 0.8132
Epoch 2/5
 - 0s - loss: 0.3698 - acc: 0.8132
Epoch 3/5
 - 0s - loss: 0.3282 - acc: 0.8132
Epoch 4/5
 - 0s - loss: 0.2816 - acc: 0.8218
Epoch 5/5
 - 0s - loss: 0.2411 - acc: 0.8707
Epoch 1/5
 - 3s - loss: 0.3327 - acc: 0.8103
Epoch 2/5
 - 1s - loss: 0.2393 - acc: 0.8764
Epoch 3/5
 - 1s - loss: 0.1792 - acc: 0.9483
Epoch 4/5
 - 1s - loss: 0.1639 - acc: 0.9253
Epoch 5/5
 - 1s - loss: 0.1465 - acc: 0.9483
Epoch 1/5
 - 2s - loss: 0.5267 - acc: 0.7989
Epoch 2/5
 - 0s - loss: 0.3651 - acc: 0.8218
Epoch 3/5
 - 1s - loss: 0.3307 - acc: 0.8218
Epoch 4/5
 - 0s - loss: 0.2932 - acc: 0.8218
Epoch 5/5
 - 0s - loss: 0.2578 - acc: 0.8420
Stacked Model: 0.9323886639676113


In [ ]:
# Stacked Model: 0.9323886639676113